In [ ]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.3 MB/s 


In [ ]:
import numpy as np
from mip import Model, xsum, maximize, BINARY, LinExpr

In [ ]:
individuals = 8
relationships = np.zeros(individuals*individuals, dtype=np.int64).reshape(individuals,individuals)

relationships[0,3] = relationships[3,0] = 1
relationships[0,1] = relationships[1,0] = -1
relationships[1,3] = relationships[3,1] = -1
relationships[1,4] = relationships[4,1] = 1
relationships[2,3] = relationships[3,2] = -1
relationships[2,4] = relationships[4,2] = -1
relationships[2,6] = relationships[6,2] = 1
relationships[2,7] = relationships[7,2] = -1
relationships[4,5] = relationships[5,4] = -1
relationships[4,6] = relationships[6,4] = 1
relationships[4,7] = relationships[7,4] = -1
relationships[6,7] = relationships[7,6] = -1

skills = np.arange(2)
teams = np.arange(2)

def demand(team, skill):
  return 1 #neste caso cada time precisa de uma pessoa para cada skill

def indSkillSet(individual):
  return np.arange(individual % 2 + 1) #individuo ímpar tem as duas skills, o par tem apenas a skill 0

def demandSkillSet(team):
  return np.arange(skills.size) #cada time precisa de uma pessoa para cada skill, logo cada time precisa de todas as skils

def indDemandSkillSet(individual, team):
  return np.intersect1d(indSkillSet(individual), demandSkillSet(team)) #retorna as skills que o time demanda e que o indíviduo possui

model = Model("CTFP")

x = [[[model.add_var(var_type=BINARY) for s in indSkillSet(u)] for u in range(individuals)] for t in teams]

model.objective = maximize(xsum( xsum( xsum( x[t][u][s] for s in indSkillSet(u) ) for u in range(individuals) ) for t in teams))

for t in teams:
  for u in range(individuals):
    for v in range(individuals):
      if relationships[u,v] == -1:
        model += xsum(x[t][u][s] for s in indSkillSet(u)) + xsum(x[t][v][s] for s in indSkillSet(v)) <= 1

for t1 in teams:
  for t2 in np.arange(t1+1, teams.size, 1):
    for u in range(individuals):
      for v in range(individuals):
        if relationships[u,v] == 1:
          model += xsum(x[t1][u][s] for s in indSkillSet(u)) + xsum(x[t2][v][s] for s in indSkillSet(v)) <= 1

for u in range(individuals):
  model += xsum( xsum( x[t][u][s] for s in indSkillSet(u)) for t in teams) <= 1

for t in teams:
  for s in demandSkillSet(t):
    model += xsum( x[t][u][s] for u in range(individuals) if s in indSkillSet(u)) >= demand(t, s)

model.optimize()

for t in teams:
  for u in range(individuals):
    for s in indSkillSet(u):
      if x[t][u][s].x == 1:
        print(f'x[{t}][{u}][{s}]')
        

x[0][1][1]
x[0][2][0]
x[0][6][0]
x[1][0][0]
x[1][3][0]
x[1][5][1]
x[1][7][0]
